In [3]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto",  torch_dtype=torch.bfloat16,  cache_dir="/media/external/siddsuresh97/huggingface/") 
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl", cache_dir="/media/external/siddsuresh97/huggingface/")
#/data



/home/siddsuresh97/packages/anaconda3/envs/conceptual_represenations_gpt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [03:08<00:00, 37.66s/it]


In [2]:
input_text = "Help me write a prompt as a question from a concept and an attribute. \nConcept: monkey\nAttribute: appears_in_comics.\nPrompt: In one word Yes/No <mask> ?"

In [10]:
input_text = "In one word Yes/No, is the feature related to the concept.\nConcept: {}\Feature: {}.\n <mask>".format("monkey", "appears_in_comics")

In [37]:
input_text = ["Q: Is the property [is_female] true for the concept [book]?\nA: False\nQ: Is the property [can_be_digital] true for the concept [book]?\nA: True\nIn one word True/False answer the following question:\nQ: Is the property [{}] true for the concept [{}]?\nA <mask>:".format("monkey", "appears_in_comics"), "Q: Is the property [is_female] true for the concept [book]?\nA: False\nQ: Is the property [can_be_digital] true for the concept [book]?\nA: True\nIn one word True/False answer the following question:\nQ: Is the property [{}] true for the concept [{}]?\nA <mask>:".format("monkey", "appears_in_comics")]

In [16]:
input_text = "Q: Is the property [is_female] true for the concept [book]?\nA: FalseQ: Is the property [can_be_digital] true for the concept [book]?A: TrueIn one word True/False answer the following question:Q: : Is the property [has two eyes] true for the concept [dolphin]?A: <mask>"

In [20]:
input_text = "Does the dolphin have two eyes? <mask>"

In [4]:
input_text = "Help me write a question from the input.\nInput - [Dolphin], [has_two_eyes]\nQuestion - Do dolphins have_two_eyes?\nInput - [piranha],[crawls]\nQuestion -"

In [5]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/home/siddsuresh97/packages/anaconda3/envs/conceptual_represenations_gpt/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Do piranhas eat crabs?</s>


In [74]:
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=10, 
    top_p=0.95, 
    num_return_sequences=20
)
# print("Output:\n" + 100 * '-')
answers = tokenizer.batch_decode(sample_outputs, skip_special_tokens=True)
# replace True with Yes and False with No
answers = [x.replace("True", "Yes").replace("False", "No") for x in answers]
# group this in batches of 20
answers = [answers[i:i+20] for i in range(0, len(answers), 20)]
# # count number of times each answer appears, do this for each list
answers = [[[x, answers[i].count(x)] for x in set(answers[i])]for i in range(len(answers))]
# sort answers by count for each list
answers = [[sorted(x, key=lambda x: x[1], reverse=True)] for x in answers]
# store the top answer and percentage of times it appeared for each list
answers = [[x[0][0][0], x[0][0][1]/20] for x in answers]
answers



[['No', 1.0], ['No', 0.7]]

In [69]:
# [sorted(x, key=lambda x: x[1], reverse=True) for x in answers[0]]
x = answers[0]
[sorted(x, key=lambda x: x[1], reverse=True)]

[[['No', 15], ['Yes', 5]]]

In [21]:
outputs

tensor([[    0, 10747,     7,    15,     1]], device='cuda:0')